In [1]:
import os
from pymongo import MongoClient, GEO2D
import pandas as pd
import json
import requests
from dotenv import load_dotenv
import re
from geopy.distance import geodesic 
load_dotenv()

True

# Limpieza base de datos y enriquecimiento con api:

In [2]:
'''startmongo = 'service mongod start'
command = 'mongoimport --db companies --collection companies --file Database/companies.json'
def initDB():
    os.popen(startmongo)
os.popen(startmongo)
os.popen('service mongod status')
os.popen(command)'''

"startmongo = 'service mongod start'\ncommand = 'mongoimport --db companies --collection companies --file Database/companies.json'\ndef initDB():\n    os.popen(startmongo)\nos.popen(startmongo)\nos.popen('service mongod status')\nos.popen(command)"

In [3]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
db,coll = (list(connectCollection('companies','companies')))
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
value =  rest[0].get('total_money_raised')

In [5]:
#q = {'$or':[{"offices.latitude":{'$ne':'null'}},{"offices.longitude":{'$ne':'null'}}]}
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
clean_db = []
for i in range(len(rest)):
    value = rest[i].get('total_money_raised') 
    valueM = re.findall(r'\d+[?\.\d]?\d',value)
    if valueM == []:
        valueM = [0]
    diction ={
        'name':rest[i]['name'],
        'category':rest[i]['category_code'],
        #'deadpooled':rest[i]['deadpooled_year'],
        'value': {
            'value':float(valueM[0]),
            'qty':value[-1]
        }
        
    }
    for ind in range(len(rest[i]['offices'])):
        if rest[i]['offices']:
            diction[f'Location-{ind+1}'] = {
                'type':'Point',
                'coordinates':[float(rest[i]['offices'][ind]['longitude']), float(rest[i]['offices'][ind]['latitude'])]
            }
        
    
    clean_db.append(diction)

In [6]:
with open('Database/database_clean.json', 'w') as fp:
    json.dump(clean_db, fp)

In [7]:
#command = 'mongoimport --db companies --collection companies_clean --jsonArray --file database_clean.json'
#os.popen(command)

In [8]:
load_dotenv()
db,coll_clean = (list(connectCollection('companies','companies_clean2')))

In [9]:
def gsearch(lat,lon,radius,query):
    URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params={
        'name':str(query),
        'location':'{},{}'.format(lat,lon),
        'radius':str(radius),
        'key':os.getenv('GOOGLE_KEY'),
        }
    res=requests.get(URL, params=params).json()
    return res

In [10]:
def geolocation(data):
    list_places = []
    for i in range(len(data['results'])):
        diction={}
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        diction = {
            'type':'Point',
            'coordinates':[lng,lat]
            }
        list_places.append(diction)
    return list_places

In [11]:
def getnearbyplaces(coll,radius,query,init = 0):
    coll_list = (list(coll.find({})))
    try:
        init = init
        for i in range(init,len(coll_list)):
            print(f'enterprise{i}')
            collitem =  coll_list[i][f'Location-1']
            lat = collitem['coordinates'][1]
            lon = collitem['coordinates'][0]
            data = gsearch(lat,lon,radius,query)
            geoloc = geolocation(data)
            locations = geoloc
            places = len(geoloc)
            value = {"$set":{'Location-1':{'type':'Point','coordinates':[float(lat), float(lon)],query:{'locations':locations,'Nº':places}}}}
            coll.update_one(coll_list[i],value)
    except KeyError as e:
        init = i+1
        print(f'Error,{e}')
        return getnearbyplaces(coll_clean,500,'starbucks',init)

In [12]:
#getnearbyplaces(coll_clean,500,'starbucks',13321)

# Limpieza Data frames:

In [13]:
coll_list = (list(coll_clean.find({})))
display(coll_list[250])




{'_id': ObjectId('5dd51184114b796302086476'),
 'name': 'BlueLithium',
 'category': 'advertising',
 'value': {'value': 11.5, 'qty': 'M'},
 'Location-1': {'type': 'Point',
  'coordinates': [37.32052, -121.948399],
  'starbucks': {'locations': [{'type': 'Point',
     'coordinates': [-121.949113, 37.320964]},
    {'type': 'Point', 'coordinates': [-121.949113, 37.320964]}],
   'Nº': 2}}}

In [14]:
transport = pd.read_csv("Database/airports-extended.csv")

In [15]:
transport = transport[transport.Type!= 'unknown']
transport

,Name,Lat,Lng,Type
0,Goroka Airport,-6.081690,145.391998,airport
1,Madang Airport,-5.207080,145.789001,airport
2,Mount Hagen Kagamuga Airport,-5.826790,144.296005,airport
3,Nadzab Airport,-6.569803,146.725977,airport
4,Port Moresby Jacksons International Airport,-9.443380,147.220001,airport
...,...,...,...,...
10651,Duisburg Hauptbahnhof,51.429402,6.774830,station
10657,Bamburi Airport,-3.979444,39.731388,airport
10658,Pioltello-Limito Railway Station,45.486111,9.329444,station
10659,Verdello-Dalmine Railway Station,45.605556,9.619722,station


In [16]:
geocode = []
for i in range(len(transport["Lat"])):
    lat = transport["Lat"].iloc[i]
    lng = transport["Lng"].iloc[i]
    diction = {
            'type':'Point',
            'coordinates':[lng,lat]
            }
    geocode.append(diction)

In [17]:
transport['geocode'] = geocode

In [18]:
from geopy.distance import geodesic 
def getdistance(location1,location2):
    return geodesic(location1, location2).km

In [19]:
db,coll = (list(connectCollection('companies','companies_clean2')))

In [20]:
transport = pd.read_csv("Database/airports-extended.csv")
transport = transport[transport.Type!= 'unknown']
geocode = []
for i in range(len(transport["Lat"])):
    lat = transport["Lat"].iloc[i]
    lng = transport["Lng"].iloc[i]
    diction = {
            'type':'Point',
            'coordinates':[lng,lat]
            }
    geocode.append(diction)
transport['geocode'] = geocode

In [21]:
def getdistance(location1,location2):
        return geodesic(location1, location2).km

In [22]:
coll_list = (list(coll.find({})))

In [23]:
data = pd.DataFrame(coll_list)
data = data[['_id','name','value','Location-1']]
data.dropna(subset=['Location-1'], inplace=True)
display(data.head())

,_id,name,value,Location-1
0,5dd51184114b79630208637c,Flektor,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.379768,..."
1,5dd51184114b79630208637d,Fox Interactive Media,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.39417, ..."
2,5dd51184114b79630208637e,Gizmoz,"{'value': 18.1, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.169472,..."
3,5dd51184114b79630208637f,Scribd,"{'value': 25.8, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.404052,..."
4,5dd51184114b796302086380,Facebook,"{'value': 2.4, 'qty': 'B'}","{'type': 'Point', 'coordinates': [-122.151801,..."


In [24]:
liststarbacks=[]
for i in range(len(data['Location-1'])):
    liststarbacks.append(data['Location-1'].iloc[i]['starbucks'])
data['Starbucks'] = liststarbacks
display(data.head())

,_id,name,value,Location-1,Starbucks
0,5dd51184114b79630208637c,Flektor,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.379768,...","{'locations': [], 'Nº': 0}"
1,5dd51184114b79630208637d,Fox Interactive Media,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.39417, ...","{'locations': [], 'Nº': 0}"
2,5dd51184114b79630208637e,Gizmoz,"{'value': 18.1, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.169472,...","{'locations': [], 'Nº': 0}"
3,5dd51184114b79630208637f,Scribd,"{'value': 25.8, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.404052,...","{'locations': [], 'Nº': 0}"
4,5dd51184114b796302086380,Facebook,"{'value': 2.4, 'qty': 'B'}","{'type': 'Point', 'coordinates': [-122.151801,...","{'locations': [], 'Nº': 0}"


In [25]:
geoindex = []
for i in range(len(data['Location-1'])):
    diction_2 = {
        'type':'Point',
        'coordinates':[data['Location-1'].iloc[i]['coordinates'][0],data['Location-1'].iloc[i]['coordinates'][1]]
    }
    geoindex.append(diction_2)
data['Location-1']=geoindex

In [26]:
db,coll_3 = connectCollection('companies','companies_clean_3')
for i in range(len(data['Location-1'])):
    latitud = data['Location-1'].iloc[i]['coordinates'][1]
    longitud = data['Location-1'].iloc[i]['coordinates'][0]
    if latitud > 90 or latitud < -90:
        data['Location-1'].iloc[i]['coordinates'][1]=longitud
        data['Location-1'].iloc[i]['coordinates'][0]=latitud
        print(f'Error, latitud = {latitud}, longitud = {longitud}')

Error, latitud = -122.402195, longitud = 37.786183
Error, latitud = -122.359397, longitud = 47.620982
Error, latitud = -118.495155, longitud = 34.0126379
Error, latitud = -121.977853, longitud = 37.374998
Error, latitud = -122.414689, longitud = 37.758113
Error, latitud = -118.756618, longitud = 34.155211
Error, latitud = -122.206893, longitud = 37.484619
Error, latitud = -122.02421, longitud = 37.36965
Error, latitud = -122.025485, longitud = 37.418531
Error, latitud = -122.401116, longitud = 37.782103
Error, latitud = -122.511687, longitud = 37.90077
Error, latitud = -95.712891, longitud = 37.09024
Error, latitud = -122.397977, longitud = 37.778076
Error, latitud = -122.4058887, longitud = 37.7791761
Error, latitud = -122.073463, longitud = 37.391502
Error, latitud = -96.771281, longitud = 32.914662
Error, latitud = -122.2013557, longitud = 37.5108541
Error, latitud = -122.404869, longitud = 37.796396
Error, latitud = -122.358484, longitud = 47.618599
Error, latitud = -122.159928, lo

Error, latitud = -95.712891, longitud = 37.09024
Error, latitud = -121.96254, longitud = 37.23697
Error, latitud = -126.557612, longitud = 54.112504
Error, latitud = -122.411498, longitud = 37.803509
Error, latitud = -122.464349, longitud = 37.652325
Error, latitud = -118.3905856, longitud = 34.0905865
Error, latitud = -118.373465, longitud = 34.065013
Error, latitud = -118.243425, longitud = 34.052187
Error, latitud = -122.3333044, longitud = 47.603176
Error, latitud = -122.012475, longitud = 37.384239
Error, latitud = -122.034021, longitud = 37.368594
Error, latitud = -122.161247, longitud = 37.445714
Error, latitud = -122.419204, longitud = 37.775196
Error, latitud = -122.4186, longitud = 37.7753
Error, latitud = -122.012475, longitud = 37.384239
Error, latitud = -122.012475, longitud = 37.384239
Error, latitud = -122.012475, longitud = 37.384239
Error, latitud = -122.302654, longitud = 37.558245
Error, latitud = -122.01235, longitud = 37.382267
Error, latitud = -95.712891, longitud

Error, latitud = -122.3564234, longitud = 47.6254784
Error, latitud = -97.4913054, longitud = 35.428318
Error, latitud = -122.0686077, longitud = 37.9715275
Error, latitud = -115.0479, longitud = 36.170206
Error, latitud = -122.391011, longitud = 37.777092
Error, latitud = -101.905099, longitud = 33.5159679
Error, latitud = 145.123041, longitud = -38.141777
Error, latitud = 174.729567, longitud = -36.74277
Error, latitud = -122.055177, longitud = 37.323234
Error, latitud = -105.278785, longitud = 40.058448
Error, latitud = -97.797117, longitud = 30.355703
Error, latitud = 103.861929, longitud = 1.308655
Error, latitud = -122.1691547, longitud = 47.5760962
Error, latitud = -111.983364, longitud = 33.58308
Error, latitud = -118.174878, longitud = 34.111471
Error, latitud = -122.421242, longitud = 37.792979
Error, latitud = -97.981336, longitud = 30.369586
Error, latitud = -118.440266, longitud = 33.987216
Error, latitud = -118.243425, longitud = 34.052187
Error, latitud = -122.347745, lo

Error, latitud = -121.9009329, longitud = 37.6621228
Error, latitud = -119.688565, longitud = 34.414177
Error, latitud = -117.7883975, longitud = 34.048489
Error, latitud = -96.694751, longitud = 40.802858
Error, latitud = 144.977417, longitud = -37.841345
Error, latitud = -116.966996, longitud = 32.653881
Error, latitud = -111.6850207, longitud = 40.311946
Error, latitud = -122.327609, longitud = 37.537542
Error, latitud = -116.5625, longitud = 43.5408
Error, latitud = -122.1842112, longitud = 37.4513629
Error, latitud = -122.1632941, longitud = 37.4441726
Error, latitud = 101.6861851, longitud = 3.0680155
Error, latitud = -93.381033, longitud = 44.983949
Error, latitud = -122.526934, longitud = 37.988395
Error, latitud = -118.474965, longitud = 33.994851
Error, latitud = -93.401984, longitud = 44.864782
Error, latitud = -122.08316, longitud = 37.388077
Error, latitud = -111.853717, longitud = 40.6110797
Error, latitud = -96.8301543, longitud = 32.9852202
Error, latitud = -122.4219405

In [27]:
db,coll_airports = connectCollection('companies','airports')
transport = transport[['Name','Type','geocode']]
for i in range(len(transport['geocode'])):
    latitud = transport['geocode'].iloc[i]['coordinates'][1]
    longitud = transport['geocode'].iloc[i]['coordinates'][0]
    if latitud > 90. or latitud < -90.:
        transport['geocode'].iloc[i]['coordinates'][1]=latitud/1000
        print(f'Error, latitud = {latitud}')
    if longitud > 180. or longitud < -180.:
        transport['geocode'].iloc[i]['coordinates'][0]=longitud/1000
        print(f'Error,longitud = {longitud}')

Error,longitud = -23965.0
Error, latitud = 46625.0
Error, latitud = 33875.0
Error,longitud = 24027.0
Error,longitud = 7387.0
Error,longitud = 9286.0
Error,longitud = 10906.0
Error,longitud = 28246.0
Error,longitud = 25655.0
Error,longitud = -1125.0
Error,longitud = 7205.0
Error, latitud = 47487.0
Error,longitud = 15535.0
Error,longitud = 13091.0
Error,longitud = 12488.0
Error, latitud = 45472.0
Error,longitud = 11255.0
Error,longitud = 12909.0
Error,longitud = 11344.0
Error,longitud = 26085.0
Error, latitud = 37979.0
Error,longitud = -107875.0
Error, latitud = 13584.0
Error,longitud = 124611.0
Error,longitud = 121125.0
Error,longitud = -60785.0
Error, latitud = -38725.0
Error,longitud = -75513.0
Error,longitud = -75294.0
Error,longitud = -72926.0
Error, latitud = 10435.0
Error,longitud = 113306.0
Error, latitud = -894.0
Error,longitud = 131287.0
Error,longitud = 142459.0
Error,longitud = -122375.0
Error,longitud = 110375.0
Error,longitud = 49375.0
Error, latitud = 25767.0
Error,longitu

In [28]:
'''
coll_3.create_index([('Location-1', pymongo.GEOSPHERE)])
coll_airports.insert_many(transport.to_dict('record'))
coll_airports.create_index ([('geocode', pymongo.GEOSPHERE)])
'''


"\ncoll_3.create_index([('Location-1', pymongo.GEOSPHERE)])\ncoll_airports.insert_many(transport.to_dict('record'))\ncoll_airports.create_index ([('geocode', pymongo.GEOSPHERE)])\n"

In [29]:
def transportloc(enterprise_data):
    numberairports = []
    locationairports= []
    for i in range(len(data['Location-1'])):
        coordinates = data['Location-1'].iloc[i]['coordinates']
        numberairports.append(len(list(coll_airports.find({'geocode': {'$geoWithin': { '$centerSphere': [ coordinates, 0.003955838970847589]}}}))))
        locationairports.append(list(coll_airports.find({'geocode': {'$geoWithin': { '$centerSphere': [ coordinates, 0.003955838970847589]}}})))
    return numberairports,locationairports

In [30]:
nairports,location = transportloc(coll_3)


In [31]:
data['N-airports'] = nairports
data['Location-airports'] = location
data.head()

,_id,name,value,Location-1,Starbucks,N-airports,Location-airports
0,5dd51184114b79630208637c,Flektor,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.379768,...","{'locations': [], 'Nº': 0}",7,"[{'_id': 5dd6a3c2dfc3b98916dff3ce, 'Name': 'Bo..."
1,5dd51184114b79630208637d,Fox Interactive Media,"{'value': 0.0, 'qty': '0'}","{'type': 'Point', 'coordinates': [-118.39417, ...","{'locations': [], 'Nº': 0}",7,"[{'_id': 5dd6a3c2dfc3b98916dfff6b, 'Name': 'Wh..."
2,5dd51184114b79630208637e,Gizmoz,"{'value': 18.1, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.169472,...","{'locations': [], 'Nº': 0}",24,"[{'_id': 5dd6a3c2dfc3b98916e0044c, 'Name': 'Br..."
3,5dd51184114b79630208637f,Scribd,"{'value': 25.8, 'qty': 'M'}","{'type': 'Point', 'coordinates': [-122.404052,...","{'locations': [], 'Nº': 0}",26,"[{'_id': 5dd6a3c2dfc3b98916e00900, 'Name': 'Ri..."
4,5dd51184114b796302086380,Facebook,"{'value': 2.4, 'qty': 'B'}","{'type': 'Point', 'coordinates': [-122.151801,...","{'locations': [], 'Nº': 0}",20,"[{'_id': 5dd6a3c2dfc3b98916e0044d, 'Name': 'Bu..."


In [32]:
locationstarbucks = []
numstarbucks = []
valuelst = []
#split Starbucks info
for i in range(len(data['Starbucks'])):    
    locationstarbucks.append(data['Starbucks'].iloc[i]['locations'])
    numstarbucks.append(data['Starbucks'].iloc[i]['Nº'])
data['Starbucks-nº'] = numstarbucks
data['Starbucks-location']  = locationstarbucks 
#split value info
for i in range(len(data['value'])):  
    dic = {
        'k':1000,
        'M':1000000,
        'B':1000000000
    }
    qty = data['value'].iloc[i]['qty']
    value = float(data['value'].iloc[i]['value'])*dic.get(qty,1)
    valuelst.append(value)
data['val']=valuelst
data = data[['_id','name','val','Location-1','Starbucks-nº','Starbucks-location','N-airports','Location-airports']]
data.columns = ['_id','Name','Value','Location','Starbucks-Nº','Starbucks-location','Airports-Nº','Airports-location']
data.head()

,_id,Name,Value,Location,Starbucks-Nº,Starbucks-location,Airports-Nº,Airports-location
0,5dd51184114b79630208637c,Flektor,0.000000e+00,"{'type': 'Point', 'coordinates': [-118.379768,...",0,[],7,"[{'_id': 5dd6a3c2dfc3b98916dff3ce, 'Name': 'Bo..."
1,5dd51184114b79630208637d,Fox Interactive Media,0.000000e+00,"{'type': 'Point', 'coordinates': [-118.39417, ...",0,[],7,"[{'_id': 5dd6a3c2dfc3b98916dfff6b, 'Name': 'Wh..."
2,5dd51184114b79630208637e,Gizmoz,1.810000e+07,"{'type': 'Point', 'coordinates': [-122.169472,...",0,[],24,"[{'_id': 5dd6a3c2dfc3b98916e0044c, 'Name': 'Br..."
3,5dd51184114b79630208637f,Scribd,2.580000e+07,"{'type': 'Point', 'coordinates': [-122.404052,...",0,[],26,"[{'_id': 5dd6a3c2dfc3b98916e00900, 'Name': 'Ri..."
4,5dd51184114b796302086380,Facebook,2.400000e+09,"{'type': 'Point', 'coordinates': [-122.151801,...",0,[],20,"[{'_id': 5dd6a3c2dfc3b98916e0044d, 'Name': 'Bu..."


# Ranking localizaciones:

In [33]:
data['Value-Rank'] = data['Value'].rank(pct=True)
data['Starbucks-Rank'] = data['Starbucks-Nº'].rank(pct=True)
data['Airports-Rank'] = data['Airports-Nº'].rank(pct=True)

rank = []
for i in range(len(data['Value-Rank'])):
    rank.append(data['Value-Rank'].iloc[i]*0.25+data['Starbucks-Rank'].iloc[i]*0.375+data['Airports-Rank'].iloc[i]*0.375)     
data['General-Rank'] = rank
data = data.sort_values(by='General-Rank', ascending=False)
data.head()

,_id,Name,Value,Location,Starbucks-Nº,Starbucks-location,Airports-Nº,Airports-location,Value-Rank,Starbucks-Rank,Airports-Rank,General-Rank
60,5dd51184114b7963020863b8,Yelp,90000000.0,"{'type': 'Point', 'coordinates': [-122.402195,...",20,"[{'type': 'Point', 'coordinates': [-122.403409...",27,"[{'_id': 5dd6a3c2dfc3b98916e00900, 'Name': 'Ri...",0.978289,0.968475,0.956103,0.966289
11642,5dd51185114b7963020890f6,Sunrun,337000000.0,"{'type': 'Point', 'coordinates': [-122.4041075...",20,"[{'type': 'Point', 'coordinates': [-122.407432...",26,"[{'_id': 5dd6a3c2dfc3b98916e00900, 'Name': 'Ri...",0.997264,0.968475,0.929039,0.960884
1854,5dd51185114b796302086aba,SquareTrade,248000000.0,"{'type': 'Point', 'coordinates': [-122.40036, ...",20,"[{'type': 'Point', 'coordinates': [-122.404304...",26,"[{'_id': 5dd6a3c2dfc3b98916e00900, 'Name': 'Ri...",0.994825,0.968475,0.929039,0.960274
305,5dd51184114b7963020864ad,OpenTable,48000000.0,"{'type': 'Point', 'coordinates': [-122.405265,...",20,"[{'type': 'Point', 'coordinates': [-122.407432...",27,"[{'_id': 5dd6a3c2dfc3b98916e00900, 'Name': 'Ri...",0.950809,0.968475,0.956103,0.959419
196,5dd51184114b796302086440,Snapfish,43500000.0,"{'type': 'Point', 'coordinates': [-122.396689,...",20,"[{'type': 'Point', 'coordinates': [-122.397592...",27,"[{'_id': 5dd6a3c2dfc3b98916e00900, 'Name': 'Ri...",0.944266,0.968475,0.956103,0.957783


In [38]:
display(data.loc[60])

_id                                            5dd51184114b7963020863b8
Name                                                               Yelp
Value                                                             9e+07
Location              {'type': 'Point', 'coordinates': [-122.402195,...
Starbucks-Nº                                                         20
Starbucks-location    [{'type': 'Point', 'coordinates': [-122.403409...
Airports-Nº                                                          27
Airports-location     [{'_id': 5dd6a3c2dfc3b98916e00900, 'Name': 'Ri...
Value-Rank                                                     0.978289
Starbucks-Rank                                                 0.968475
Airports-Rank                                                  0.956103
General-Rank                                                   0.966289
Name: 60, dtype: object

# Folium:

In [119]:
import folium
import webbrowser

In [120]:
company = data.loc[60]
coordinates = company['Location']['coordinates'][::-1]
Map = folium.Map(location=coordinates,width=950, height=500, zoom_start=14)
folium.Marker(coordinates,
                        radius=4,
                        icon=folium.Icon(color='blue',icon='hand-down'),
                        popup="Our Company!",
                        fill_color="#F35C50"
                       ).add_to(Map)
for i in range(len(company['Starbucks-location'])):
    coords = company['Starbucks-location'][i]['coordinates'][::-1]
    folium.Marker(coords,
                        radius=4,
                        popup="Starbucks",
                        icon=folium.Icon(color='green',icon='tower'),
                        fill_color="#F35C50",
                       ).add_to(Map)
for i in range(len(company['Airports-location'])):
    name = company['Airports-location'][i]['Name']
    coords = company['Airports-location'][i]['geocode']['coordinates'][::-1]
    if bool(re.findall(r'Station',name)):
        folium.Marker(coords,
                        radius=4,
                        popup=name,
                        icon=folium.Icon(color='red',icon='arrow-down'),
                        fill_color="#F35C50",
                       ).add_to(Map)
    else:
        folium.Marker(coords,
                        radius=4,
                        popup=name,
                        icon=folium.Icon(color='white',icon='send'),
                        fill_color="#000000",
                       ).add_to(Map)
Map.save('Map.html')
webbrowser.open('Map.html', new=2)
Map